In [5]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import classification_report

In [9]:
def segment_users(row, median_sessions,median_sessions_2,median_sessions_3,median_sessions_4):
    """ Segment users based on engagement levels"""
    if row['sessions'] > median_sessions and row['total_navigations_fav1'] >= median_sessions_2 and row['n_days_after_onboarding'] >= median_sessions_3 and row['drives']<=median_sessions_4:
        return 'High Engagement'
    else:
        return 'Low Engagement'

In [10]:
def segment_driving_days(row, median_sessions,median_sessions_2):
    """ Segment users based on day levels"""
    if row['activity_days'] <= median_sessions and row['driving_days'] <= median_sessions_2 :
        return 'High day'
    else:
        return 'Low day'

In [15]:
# Load the scaler and model
with open('/content/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('/content/SelectFromModel.pkl', 'rb') as file:
    selector = pickle.load(file)

with open('/content/logistic_regression_model.pkl', 'rb') as file:
    logistic_regression_model = pickle.load(file)



In [ ]:
def preprocess_dataframe(df):
    """Preprocess the input DataFrame."""
    columns_to_drop = ['ID', 'device']
    existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    df.drop(columns=existing_columns_to_drop, inplace=True)
    return df

In [37]:
def make_predictions(file=None, sessions=None, drives=None, total_sessions=None,
                     n_days_after_onboarding=None, total_navigations_fav1=None,
                     total_navigations_fav2=None, driven_km_drives=None,
                     duration_minutes_drives=None, activity_days=None,
                     driving_days=None, device=None):

    if file is not None:
        df = pd.read_csv(file.name)
        X_test = df.drop('label', axis=1, errors='ignore')
        y_test = df.get('label', None)
    else:
        columns = ['sessions', 'drives', 'total_sessions', 'n_days_after_onboarding',
                   'total_navigations_fav1', 'total_navigations_fav2',
                   'driven_km_drives', 'duration_minutes_drives',
                   'activity_days', 'driving_days', 'device']

        input_data = [sessions, drives, total_sessions, n_days_after_onboarding,
                      total_navigations_fav1, total_navigations_fav2,
                      driven_km_drives, duration_minutes_drives,
                      activity_days, driving_days, device]

        df = pd.DataFrame([input_data], columns=columns)
        X_test = df

    X_test = preprocess_dataframe(X_test)

    # Segmenting engagement levels and day levels
    X_test['engagement_level'] = X_test.apply(segment_users, axis=1, args=(
        X_test['sessions'].median(),
        X_test['total_navigations_fav1'].median(),
        X_test['n_days_after_onboarding'].median(),
        X_test['drives'].median()
    ))

    X_test['day_level'] = X_test.apply(segment_driving_days, axis=1, args=(
        X_test['activity_days'].median(),
        X_test['driving_days'].median()
    ))

    # Calculating new features
    X_test['activity_ratio'] = X_test['driving_days'] / X_test['activity_days'].replace(0, np.nan)
    X_test['avg_distance_per_drive'] = X_test['driven_km_drives'] / X_test['drives'].replace(0, np.nan)
    X_test['engagement_ratio'] = X_test['total_sessions'] / X_test['driving_days'].replace(0, np.nan)
    X_test['avg_navigations_fav'] = (X_test['total_navigations_fav1'] + X_test['total_navigations_fav2']) / 2

    # Filling NaN values
    for col in ['activity_ratio', 'avg_distance_per_drive', 'engagement_ratio', 'avg_navigations_fav']:
        X_test[col].fillna(X_test[col].median(), inplace=True)

    # Mapping engagement and day levels to numerical values
    X_test['engagement_level'] = X_test['engagement_level'].map({'Low Engagement': 0, 'High Engagement': 1})
    X_test['day_level'] = X_test['day_level'].map({'Low day': 0, 'High day': 1})


    # Prepare the DataFrame for scaling
    temp_X_test = X_test.copy()

    # Scale the necessary features
    temp_X_test['n_days_after_onboarding'] = (X_test['n_days_after_onboarding'] / 365).astype(float)
    temp_X_test['duration_minutes_drives'] = (X_test['duration_minutes_drives'] / (60 * 24)).astype(float)

    columns = ['total_navigations_fav1','total_navigations_fav2','total_sessions','driven_km_drives']

    temp_X_test[columns] = scaler.transform(temp_X_test[columns])

    # Feature Selection
    X_test_selected = selector.transform(temp_X_test)

    # Make predictions
    y_pred = logistic_regression_model.predict(X_test_selected)
    y_pred_label = ['churned' if pred == 0 else 'retained' for pred in y_pred]

    # Prepare results
    if file is not None:
        if y_test is not None:
            results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_label})
            report = classification_report(y_test, y_pred_label, output_dict=True)
            return results, report
        else:
            results = pd.DataFrame({'Actual': [None], 'Predicted': [y_pred_label[0]]})
            report = {"Prediction": y_pred_label[0]}
            return results, report
    else:
        results = pd.DataFrame({'Actual': [None], 'Predicted': [y_pred_label[0]]})
        report = {"Prediction": y_pred_label[0]}
        return results, report


In [40]:
iface = gr.Interface(
    fn=make_predictions,
    inputs=[
        gr.File(label="Upload CSV File")
    ],
    outputs=[
        gr.DataFrame(label="Results"),
        gr.JSON(label="Custom Report")
    ],
    title="Waze App",
    description="Predict user churn for the Waze app. You can upload a CSV file with multiple user records."
)

iface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fbc20fb18f19ad3ffb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fbc20fb18f19ad3ffb.gradio.live


In [41]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 8.9 MB/s eta 0:00:00


In [42]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import classification_report

# Load the scaler and model
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('SelectFromModel.pkl', 'rb') as file:
    selector = pickle.load(file)

with open('logistic_regression_model.pkl', 'rb') as file:
    logistic_regression_model = pickle.load(file)

def segment_users(row, median_sessions, median_sessions_2, median_sessions_3, median_sessions_4):
    """ Segment users based on engagement levels"""
    if (row['sessions'] > median_sessions and
        row['total_navigations_fav1'] >= median_sessions_2 and
        row['n_days_after_onboarding'] >= median_sessions_3 and
        row['drives'] <= median_sessions_4):
        return 'High Engagement'
    else:
        return 'Low Engagement'

def segment_driving_days(row, median_sessions, median_sessions_2):
    """ Segment users based on day levels"""
    if row['activity_days'] <= median_sessions and row['driving_days'] <= median_sessions_2:
        return 'High day'
    else:
        return 'Low day'

def preprocess_dataframe(df):
    """Preprocess the input DataFrame."""
    columns_to_drop = ['ID', 'device']
    existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    df.drop(columns=existing_columns_to_drop, inplace=True)
    return df

def make_predictions(df):
    X_test = preprocess_dataframe(df)

    # Segmenting engagement levels and day levels
    X_test['engagement_level'] = X_test.apply(segment_users, axis=1, args=(
        X_test['sessions'].median(),
        X_test['total_navigations_fav1'].median(),
        X_test['n_days_after_onboarding'].median(),
        X_test['drives'].median()
    ))

    X_test['day_level'] = X_test.apply(segment_driving_days, axis=1, args=(
        X_test['activity_days'].median(),
        X_test['driving_days'].median()
    ))

    # Calculating new features
    X_test['activity_ratio'] = X_test['driving_days'] / X_test['activity_days'].replace(0, np.nan)
    X_test['avg_distance_per_drive'] = X_test['driven_km_drives'] / X_test['drives'].replace(0, np.nan)
    X_test['engagement_ratio'] = X_test['total_sessions'] / X_test['driving_days'].replace(0, np.nan)
    X_test['avg_navigations_fav'] = (X_test['total_navigations_fav1'] + X_test['total_navigations_fav2']) / 2

    # Filling NaN values
    for col in ['activity_ratio', 'avg_distance_per_drive', 'engagement_ratio', 'avg_navigations_fav']:
        X_test[col].fillna(X_test[col].median(), inplace=True)

    # Mapping engagement and day levels to numerical values
    X_test['engagement_level'] = X_test['engagement_level'].map({'Low Engagement': 0, 'High Engagement': 1})
    X_test['day_level'] = X_test['day_level'].map({'Low day': 0, 'High day': 1})

    # Prepare the DataFrame for scaling
    temp_X_test = X_test.copy()

    # Scale the necessary features
    temp_X_test['n_days_after_onboarding'] = (X_test['n_days_after_onboarding'] / 365).astype(float)
    temp_X_test['duration_minutes_drives'] = (X_test['duration_minutes_drives'] / (60 * 24)).astype(float)

    columns = ['total_navigations_fav1', 'total_navigations_fav2', 'total_sessions', 'driven_km_drives']

    temp_X_test[columns] = scaler.transform(temp_X_test[columns])

    # Feature Selection
    X_test_selected = selector.transform(temp_X_test)

    # Make predictions
    y_pred = logistic_regression_model.predict(X_test_selected)
    y_pred_label = ['churned' if pred == 0 else 'retained' for pred in y_pred]

    return pd.DataFrame({'Predicted': y_pred_label})

# Streamlit app
st.title("Waze App User Churn Prediction")
st.write("Upload a CSV file with user records for churn prediction.")

# File uploader
uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    st.write("Data Preview:")
    st.dataframe(df.head())

    # Make predictions
    results = make_predictions(df)
    st.write("Prediction Results:")
    st.dataframe(results)


2024-10-22 12:21:17.381 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 12:21:17.497 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-22 12:21:17.500 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 12:21:17.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 12:21:17.505 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 12:21:17.507 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 12:21:17.509 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 12:21:17.511 Thread 'MainThread': mi